In [1]:
# Import packages
import pandas as pd 
import numpy as np 
import geopandas as gpd 

from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import io
import json

## Downloading all Snuffelfiets data

In [46]:
url = 'https://ckan.dataplatform.nl/api/3/action/package_show?id=9cc4de28-6d03-4b59-8c66-085b3e8b3956&include_tracking=true'

r = requests.get(url)

data = r.json()

In [112]:
csv_urls = [x['url'] for x in data['result']['resources']]
csv_urls[0]

'https://ckan.dataplatform.nl/dataset/9cc4de28-6d03-4b59-8c66-085b3e8b3956/resource/a12b08b4-d0bf-45f9-b17c-20448369754c/download/resource_2020_05_11_2020_05_18.csv'

In [119]:
test_csv_urls = csv_urls[:3]

### Warning: running the block below will result in downloading ~100 .csv's


In [118]:
data = []

for csv in tqdm(test_csv_urls): 
    response = requests.get(csv)
    file_object = io.StringIO(response.content.decode('utf-8'))
    df = pd.read_csv(file_object)
    data.append(df)


df = pd.concat(data)    # merge all csv's
df.reset_index(drop=True, inplace=True)
df.to_csv("../data/external/all_snuffelfiets_raw.csv", index=False)
data = []   # clear memory

100%|██████████| 5/5 [00:39<00:00,  7.98s/it]


In [120]:
snuffel_all = pd.read_csv("../data/external/all_snuffelfiets_raw.csv")

In [121]:
snuffel_all.head()

,sensor,air_quality_observed_id,lon,lat,recording_time,trip_sequence,humidity,pm10,pm1_0,pm2_5,pressure,temperature,voc,voltage,error_code,version_major,version_minor,acc_max,no2
0,c432f4d9cac0cc68e786d054c1ec771f,31925610,4.830358,52.017010,2020-05-18 11:59:19,19,57,4,4,4,1049,23.4,188,3.81,0,1,7,0,0
1,74678958154e0d1a078e5a6f50b73b73,31925642,5.057220,52.023201,2020-05-18 11:59:19,16,55,6,6,6,1042,21.5,190,4.06,0,1,7,0,0
2,7d320c4527e25b06c3c2cfc68c917019,31925576,5.237413,52.023979,2020-05-18 11:59:16,39,50,9,8,8,1054,24.2,201,4.02,0,1,7,0,0
3,c432f4d9cac0cc68e786d054c1ec771f,31925609,4.829355,52.017056,2020-05-18 11:59:08,19,57,4,4,4,1049,23.3,187,3.81,0,1,7,0,0
4,74678958154e0d1a078e5a6f50b73b73,31925640,5.056912,52.022751,2020-05-18 11:59:07,16,55,7,6,7,1042,21.5,189,4.06,0,1,7,1,0


In [122]:
csv_urls

['https://ckan.dataplatform.nl/dataset/9cc4de28-6d03-4b59-8c66-085b3e8b3956/resource/a12b08b4-d0bf-45f9-b17c-20448369754c/download/resource_2020_05_11_2020_05_18.csv',
 'https://ckan.dataplatform.nl/dataset/9cc4de28-6d03-4b59-8c66-085b3e8b3956/resource/df752c8d-a9a6-4f94-b1ee-2d24e623c343/download/resource_2020_05_04_2020_05_11.csv',
 'https://ckan.dataplatform.nl/dataset/9cc4de28-6d03-4b59-8c66-085b3e8b3956/resource/b9cf1bc9-d348-4663-9ab1-c4bab32e8bbb/download/resource_2020_04_27_2020_05_04.csv',
 'https://ckan.dataplatform.nl/dataset/9cc4de28-6d03-4b59-8c66-085b3e8b3956/resource/13035a1f-59e0-4499-9fbe-5c7a6dbb15de/download/resource_2020_04_20_2020_04_27.csv',
 'https://ckan.dataplatform.nl/dataset/9cc4de28-6d03-4b59-8c66-085b3e8b3956/resource/66571e1a-2567-4a0c-b077-c4a438d309a9/download/resource_2020_04_13_2020_04_20.csv',
 'https://ckan.dataplatform.nl/dataset/9cc4de28-6d03-4b59-8c66-085b3e8b3956/resource/91816929-78ab-45a1-8b5e-1a9c46b575e1/download/resource_2020_04_06_2020_04_1